In [1]:
import time
from pathlib import Path

from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from trulens_eval import Tru, TruChain, FeedbackMode
from trulens_eval.instruments import Instrument
from trulens_eval.sessions import SessionManager
from trulens_eval.messages import OpenAiFunctionsMessagesExtractor

In [2]:
tru = Tru()

In [3]:
tru.start_dashboard(force=True)

Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.178.30:8501 .


In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="Nothing",
        func=lambda x: x,
        description="useful for for nothing"
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
Instrument.Default.CLASSES.add(agent.agent.__class__)
Instrument.Default.MODULES.add(agent.agent.__module__)
Instrument.Default.METHODS["plan"] = lambda o: isinstance(o, agent.agent.__class__)

In [ ]:
app = TruChain(
    app=agent,
    app_id="OpenAiFunctions",
    feedback_mode=FeedbackMode.WITH_APP,
)

In [ ]:
session_manager = SessionManager(
    app=app,
    messages_extractor=OpenAiFunctionsMessagesExtractor.from_tools(tools)
)

In [ ]:
queries = [
    "what is the square root of 123",
    "how much is Euler's number divided by PI"
]

with session_manager.start_session() as session_runner:
    for query in queries:
        session_runner(inputs={"input": query})

In [ ]:
session_runner.get_messages() # messages extracted from the last session

In [ ]:
session_manager.get_messages()  # messages extracted from all sessions